In [1]:
import numpy as np
from paik.solver import Solver, DEFAULT_SOLVER_PARAM_M7_NORM

solver = Solver(solver_param=DEFAULT_SOLVER_PARAM_M7_NORM)

WorldModel::LoadRobot: /home/luca/.cache/jrl/temp_urdfs/panda_arm_hand_formatted_link_filepaths_absolute.urdf
joint mimic: no multiplier, using default value of 1 
joint mimic: no offset, using default value of 0 
URDFParser: Link size: 17
URDFParser: Joint size: 12
LoadAssimp: Loaded model /home/luca/miniconda3/lib/python3.9/site-packages/jrl/urdfs/panda/meshes/visual/link0.dae (59388 verts, 20478 tris)
LoadAssimp: Loaded model /home/luca/miniconda3/lib/python3.9/site-packages/jrl/urdfs/panda/meshes/visual/link1.dae (37309 verts, 12516 tris)
LoadAssimp: Loaded model /home/luca/miniconda3/lib/python3.9/site-packages/jrl/urdfs/panda/meshes/visual/link2.dae (37892 verts, 12716 tris)
LoadAssimp: Loaded model /home/luca/miniconda3/lib/python3.9/site-packages/jrl/urdfs/panda/meshes/visual/link3.dae (42512 verts, 14233 tris)
LoadAssimp: Loaded model /home/luca/miniconda3/lib/python3.9/site-packages/jrl/urdfs/panda/meshes/visual/link4.dae (43520 verts, 14620 tris)
LoadAssimp: Loaded model /ho

In [2]:
num_im = 100
num_ta = 100
num_sols = 1

solver.shrink_ratio = 0.0
J_im, P_im, F_im = solver.random_sample_JPF(num_samples=num_im)
J_ta, P_ta, F_ta = solver.random_sample_JPF(num_samples=num_ta)

In [3]:
l2_errs = np.zeros((num_im, num_ta))
ang_errs = np.zeros((num_im, num_ta))
config_errs = np.zeros((num_im, num_ta))

for i, (j_im, f_im) in enumerate(zip(J_im, F_im)):
    F = np.full_like(F_ta, *f_im)
    J = solver.solve(P=P_ta, F=F, num_sols=num_sols, return_numpy=True)
    l2_errs[i], ang_errs[i] = solver.evaluate_solutions(J, P_ta, return_col=True)
    config_errs[i] = np.linalg.norm(J - j_im, axis=2).mean(axis=0)

In [4]:
import pandas as pd

df = pd.DataFrame(
    {
        "l2_err": l2_errs.flatten(),
        "ang_err": ang_errs.flatten(),
        "config_err": config_errs.flatten(),
    }
)

df.describe()

,l2_err,ang_err,config_err
count,10000.000000,10000.000000,10000.000000
mean,0.005530,0.061088,5.058875
std,0.022582,0.186925,1.346186
min,0.000152,0.000977,1.206840
25%,0.002017,0.013602,4.095296
50%,0.003198,0.022772,5.052333
75%,0.004871,0.040101,6.008044
max,1.175009,3.123437,9.804457


In [5]:
import numpy as np
import pandas as pd
from ikflow.utils import set_seed
from ikflow.model_loading import get_ik_solver

set_seed()

# Build IKFlowSolver and set weights
ik_solver, hyper_parameters = get_ik_solver("panda__full__lp191_5.25m")
robot = ik_solver.robot

# -> unrefined solutions
l2_errs = np.zeros((num_im, num_ta))
ang_errs = np.zeros((num_im, num_ta))
config_errs = np.zeros((num_im, num_ta))

for i, p in enumerate(P_ta):
    (
        solutions,
        l2_errors,
        angular_errors,
        _,
        _,
        _,
    ) = ik_solver.solve(
        p, n=num_im, refine_solutions=False, return_detailed=True
    )  # type: ignore

    solutions = solutions.detach().cpu().numpy()

    l2_errs[:, i] = l2_errors
    ang_errs[:, i] = angular_errors
    
    for j, j_im in enumerate(J_im):
        config_errs[j, i] = np.linalg.norm(solutions - j_im, axis=1).mean()

config.py: Using device 'cuda:0'
set_seed() - random int:  541
WorldModel::LoadRobot: /tmp/panda_arm_hand_formatted_link_filepaths_absolute.urdf
ndim_tot=7
dim_cond=8
joint mimic: no multiplier, using default value of 1 
joint mimic: no offset, using default value of 0 
URDFParser: Link size: 17
URDFParser: Joint size: 12
LoadAssimp: Loaded model /home/luca/miniconda3/lib/python3.9/site-packages/jkinpylib/urdfs/panda/meshes/visual/link0.dae (59388 verts, 20478 tris)
LoadAssimp: Loaded model /home/luca/miniconda3/lib/python3.9/site-packages/jkinpylib/urdfs/panda/meshes/visual/link1.dae (37309 verts, 12516 tris)
LoadAssimp: Loaded model /home/luca/miniconda3/lib/python3.9/site-packages/jkinpylib/urdfs/panda/meshes/visual/link2.dae (37892 verts, 12716 tris)
LoadAssimp: Loaded model /home/luca/miniconda3/lib/python3.9/site-packages/jkinpylib/urdfs/panda/meshes/visual/link3.dae (42512 verts, 14233 tris)
LoadAssimp: Loaded model /home/luca/miniconda3/lib/python3.9/site-packages/jkinpylib/urd

In [6]:
df = pd.DataFrame(
    {
        "l2_err": l2_errs.flatten(),
        "ang_err": ang_errs.flatten(),
        "config_err": config_errs.flatten(),
    }
)

df.describe()

,l2_err,ang_err,config_err
count,10000.000000,10000.000000,10000.000000
mean,0.007875,0.047500,5.149393
std,0.018297,0.165467,0.781112
min,0.000099,0.000488,2.418903
25%,0.002475,0.009003,4.604053
50%,0.003862,0.013978,5.131788
75%,0.006315,0.025262,5.673380
max,0.493876,3.063786,8.429960
